# API-info

In [2]:
from openai import OpenAI
import httpx
client = OpenAI(
    base_url="https://api.xiaoai.plus/v1",
    api_key="sk-gjmt4deRRB00reuN10Af64DdBeDd4a8e8b45Fb58Bd57A631",
    http_client=httpx.Client(
        base_url="https://api.xiaoai.plus/v1",
        follow_redirects=True,
    ),
)

# Zero-shot

In [3]:
completion = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content": (
            "You will be provided with two sentences."
            "Your task is to rewrite the given sentences respectively without changing their meanings."
            "Keep all the pronouns you encounter in the first sentence. There's no such constraint for the second one."
            
            "Output format:"
            "Sentence one rewrite: (end with '\n')"
            "Sentence two rewrite:"
            
        )},
        {"role": "user", "content": 
            "sentence one: I stuck a pin through a carrot. When I pulled the pin out, it had a hole."
            "sentence two: The carrot had a hole."}
    ]
)

print(completion.choices[0].message.content)

Sentence one rewrite: I stuck a pin through a carrot. When I pulled it out, there was a hole.  
Sentence two rewrite: The carrot had a hole.


结论：即便让它保留代词，也只是形式上的保留，有更改代词指代对象的风险，需要进一步指定保留规则。

# CoT with zero-shot

In [3]:
completion = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content": (
            "You will be provided with two sentences."
            "Your task is to rewrite the given sentences respectively without changing their meanings."
            "Keep all the pronouns you encounter in the first sentence. There's no such constraint for the second sentence."
            
            "Always remember that the two sentence are unrelated, so do not refer to each other when rewriting."
            
            "first, find all the pronouns in the first sentence."
            "second, find the object to which the pronoun refers in the first sentence."
            "third, rewrite the context except the pronouns in the first sentence without any reference to the second sentence, keep all the original pronouns there."
            "fourth, check whether you changed any of the object (to which the pronoun refers) that you have found in second step in the first sentence, if so restart from the third step."
            "fifth, only output the content of your last rewrite version with the following format."
            "at last, rewrite the second sentence without any reference to the first sentence."
            
            "Output format:"
            "You should output two lines, with each rewrite sentence at one line."
            
        )},
        {"role": "user", "content": 
            "sentence one: The firemen arrived after the police because they were coming from so far away."
            "sentence two: The police were coming from so far away."}
    ]
)

print(completion.choices[0].message.content)

The firemen arrived after the police because they were coming from so far away.  
The police arrived from a long distance.


# 测试总结

- 判断代词指代是否正确（第一句中含有代词）

- 由于测试前提是第一个句子中含有代词，故而要求模型保留所有代词
  
- 由于需要分步骤，所以每一步任务都是单独的而不是混合
  
- 依然没有使用few-shot，认为这样仍会限制模型的改写方式

- 两句话之间可能会相互参考，导致第二句话的代词被第一句话中的对象替代掉，这回影响结果判断，所以严格要求独立对待两句话，效果明显
  